In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
from sklearn.cluster import KMeans 
from geopy.geocoders import Nominatim 
import matplotlib.cm as cm
import matplotlib.colors as colors

In [3]:
!conda install -c conda-forge folium --yes
import folium

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - folium


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.2               |     pyhd8ed1ab_0          26 KB  conda-forge
    certifi-2021.5.30          |   py37h89c1867_0         141 KB  conda-forge
    folium-0.12.0              |     pyhd8ed1ab_1          64 KB  conda-forge
    openssl-1.1.1k             |       h7f98852_0         2.1 MB  conda-forge
    python_abi-3.7             |          2_cp37m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.3 MB

The following NEW packages will be INSTALLED:

  branca             conda-forge/noarch::branca-0.4.2-pyhd8ed1ab_0
  folium             conda-forge/noarch::folium-0.12.0-pyhd8ed1ab_1
  pytho

In [4]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_data = requests.get(url).text
soup = BeautifulSoup(html_data, 'html5lib')

In [5]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [6]:
for tr_cell in table.find_all('tr'):
    row_data=[]
    for td_cell in tr_cell.find_all('td'):
        row_data.append(td_cell.text.strip())
    if len(row_data)==3:
        df.loc[len(df)] = row_data
        df.head()

In [7]:
print(f"There are {df.loc[df['Borough'] == 'Not assigned'].shape[0]} Not assigned Boroughs")


There are 0 Not assigned Boroughs


In [8]:
print(f"There are {df.loc[df['Neighborhood'] == 'Not assigned'].shape[0]} Not assigned Neighborhoods")


There are 0 Not assigned Neighborhoods


In [9]:
df.head()



,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


In [10]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]


,PostalCode,Borough,Neighborhood
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
6,M1B,Scarborough,"Malvern, Rouge"
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
23,M4G,East York,Leaside
24,M5G,Downtown Toronto,Central Bay Street
27,M2H,North York,Hillcrest Village
32,M1J,Scarborough,Scarborough Village
50,M9L,North York,Humber Summit
54,M4M,East Toronto,Studio District
71,M1R,Scarborough,"Wexford, Maryvale"


In [11]:
df.shape


(103, 3)

In [13]:
 !conda install -c conda-forge geocoder --yes
 import geocoder

 for index, content in df.iterrows():
     coordinates = None
     while(coordinates is None):
         coordinates = geocoder.arcgis(f'{content[0]}, Toronto, Ontario').latlng
     latitude = coordinates[0]
     longitude = coordinates[1]
     df.loc[index, 'Latitude'] = latitude
     df.loc[index, 'Longitude'] = longitude

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python-3.7-OpenCE

  added / updated specs:
    - geocoder


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geocoder-1.38.1            |             py_1          53 KB  conda-forge
    ratelim-0.1.6              |             py_2           6 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          59 KB

The following NEW packages will be INSTALLED:

  geocoder           conda-forge/noarch::geocoder-1.38.1-py_1
  ratelim            conda-forge/noarch::ratelim-0.1.6-py_2



geocoder-1.38.1      | 53 KB     | ##################################### | 100% 
ratelim-0.1.6        | 6 KB      | ##################################### | 100% 
Preparing transaction: done
Verifying transaction: done
Executing transaction: done


In [21]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188


In [22]:
example_postalcode = ["M5G", "M2H", "M4B", "M1J", "M4G", "M4M", "M1R", "M9V", "M9L", "M5V", "M1B", "M5A"]

df.loc[df['PostalCode'].isin(example_postalcode)]

,PostalCode,Borough,Neighborhood,Latitude,Longitude
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
6,M1B,Scarborough,"Malvern, Rouge",43.81139,-79.19662
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.70718,-79.31192
23,M4G,East York,Leaside,43.70902,-79.36349
24,M5G,Downtown Toronto,Central Bay Street,43.65609,-79.38493
27,M2H,North York,Hillcrest Village,43.80225,-79.35558
32,M1J,Scarborough,Scarborough Village,43.74446,-79.23117
50,M9L,North York,Humber Summit,43.75948,-79.55707
54,M4M,East Toronto,Studio District,43.66213,-79.33497
71,M1R,Scarborough,"Wexford, Maryvale",43.75075,-79.30054


In [23]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [24]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [25]:
df_toronto_borough = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto_borough.shape

(39, 5)

In [26]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_toronto_borough['Latitude'], df_toronto_borough['Longitude'], df_toronto_borough['Borough'], df_toronto_borough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [27]:
CLIENT_ID = 'JFSS012WP5IME1FS05UJAEPV0KQ2JV0210RFBG4PMEUF01AT' # your Foursquare ID
CLIENT_SECRET = 'VPUAG5M2XS4LLQYXOL5AV3KSHPKU2O4D30FDDS5A1VHXJCBX' # your Foursquare Secret
VERSION = '20210818' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JFSS012WP5IME1FS05UJAEPV0KQ2JV0210RFBG4PMEUF01AT
CLIENT_SECRET:VPUAG5M2XS4LLQYXOL5AV3KSHPKU2O4D30FDDS5A1VHXJCBX


In [28]:
df_toronto_borough.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.65739,-79.37804
2,M5C,Downtown Toronto,St. James Town,43.65215,-79.37587
3,M4E,East Toronto,The Beaches,43.67709,-79.29547
4,M5E,Downtown Toronto,Berczy Park,43.64536,-79.37306


In [29]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [30]:
borough_neighborhood_venues = getNearbyVenues(boroughs=df_toronto_borough['Borough'],
                                             neighborhoods=df_toronto_borough['Neighborhood'],
                                             latitudes=df_toronto_borough['Latitude'],
                                             longitudes=df_toronto_borough['Longitude'])

In [31]:
borough_neighborhood_venues.head()

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot
3,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,The Yoga Lounge,43.655515,-79.364955,Yoga Studio
4,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Berkeley Church,43.655123,-79.365873,Event Space


In [32]:
borough_neighborhood_venues.shape

(1721, 8)

In [33]:
borough_neighborhood_venues.groupby(['Borough', 'Neighborhood']).count()

Neighborhood Latitude  \
Borough                Neighborhood                                                                
Central Toronto        Davisville                                                             27   
                       Davisville North                                                        8   
                       Forest Hill North & West                                                1   
                       Lawrence Park                                                           2   
                       Moore Park, Summerhill East                                             4   
                       North Toronto West                                                      3   
                       Roselawn                                                                1   
                       Summerhill West, Rathnelly, South Hill, Forest ...                      6   
                       The Annex, North Midtown, Yorkville                                    23   
Downtown Toronto       Berczy Park                                                            63   
                       CN Tower, King and Spadina, Railway Lands, Harb...                     75   
                       Central Bay Street                                                     72   
                       Christie                                                               11   
                       Church and Wellesley                                                   84   
                       Commerce Court, Victoria Hotel                                        100   
                       First Canadian Place, Underground city                                100   
                       Garden District, Ryerson                                              100   
                       Harbourfront East, Union Station, Toronto Islands                      56   
                       Kensington Market, Chinatown, Grange Park                              52   
                       Regent Park, Harbourfront                                              22   
                       Richmond, Adelaide, King                                              100   
                       Rosedale                                                                4   
                       St. James Town                                                         83   
                       St. James Town, Cabbagetown                                            45   
                       Toronto Dominion Centre, Design Exchange                              100   
                       University of Toronto, Harbord                                         51   
Downtown Toronto Stn A Enclave of M5E                                                        100   
East Toronto           India Bazaar, The Beaches West                                         21   
                       Studio District                                                        50   
                       The Beaches                                                             5   
                       The Danforth West, Riverdale                                            8   
East Toronto Business  Enclave of M4L                                                        100   
East York/East Toronto The Danforth  East                                                      4   
West Toronto           Brockton, Parkdale Village, Exhibition Place                           82   
                       Dufferin, Dovercourt Village                                           19   
                       High Park, The Junction South                                           5   
                       Little Portugal, Trinity                                               43   
                       Parkdale, Roncesvalles                                                 52   
                       Runnymede, Swansea                                  

In [34]:
print(f'There are {borough_neighborhood_venues["Venue Category"].nunique()} uniques categories.')


There are 225 uniques categories.


In [35]:
# one hot encoding
toronto_onehot = pd.get_dummies(borough_neighborhood_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = borough_neighborhood_venues['Borough'] 
toronto_onehot['Neighborhoods'] = borough_neighborhood_venues['Neighborhood'] 
toronto_onehot['Venue'] = borough_neighborhood_venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Borough,Neighborhoods,Venue,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Figs Breakfast & Lunch,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Downtown Toronto,"Regent Park, Harbourfront",The Yoga Lounge,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,Downtown Toronto,"Regent Park, Harbourfront",Berkeley Church,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
toronto_onehot.shape


(1721, 228)

In [37]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped

,Borough,Neighborhoods,Accessories Store,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,...,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.037037,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Central Toronto,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Central Toronto,North Toronto West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Central Toronto,Roselawn,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Central Toronto,"The Annex, North Midtown, Yorkville",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Downtown Toronto,Berczy Park,0.000000,0.000000,0.015873,0.000000,0.015873,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.015873,0.000000,0.000000,0.000000,0.000000,0.000000,0.015873


In [38]:
toronto_grouped.shape


(39, 227)

In [39]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.11
1                Café  0.07
2         Pizza Place  0.07
3  Italian Restaurant  0.07
4         Coffee Shop  0.07


----Davisville North----
                  venue  freq
0              Bus Line  0.12
1     Food & Drink Shop  0.12
2        Breakfast Spot  0.12
3                  Park  0.12
4  Gym / Fitness Center  0.12


----Forest Hill North & West----
                             venue  freq
0                             Park   1.0
1                Accessories Store   0.0
2                     Noodle House   0.0
3       Modern European Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Lawrence Park----
                venue  freq
0         Swim School   0.5
1            Bus Line   0.5
2   Accessories Store   0.0
3           Nightclub   0.0
4  Miscellaneous Shop   0.0


----Moore Park, Summerhill East----
               venue  freq
0  Convenience Store  0.25
1       Intersection  0.25
2             

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [41]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,Dessert Shop,Italian Restaurant,Café,Sandwich Place,Pizza Place,Coffee Shop,Gym,Thai Restaurant,Movie Theater,Sushi Restaurant
1,Central Toronto,Davisville North,Bus Line,Hotel,Breakfast Spot,Park,Food & Drink Shop,Department Store,Gym / Fitness Center,Playground,Fish Market,Fish & Chips Shop
2,Central Toronto,Forest Hill North & West,Park,Yoga Studio,Dumpling Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm,Falafel Restaurant
3,Central Toronto,Lawrence Park,Bus Line,Swim School,Yoga Studio,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm
4,Central Toronto,"Moore Park, Summerhill East",Convenience Store,Gym,Intersection,Restaurant,Yoga Studio,Dry Cleaner,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Farm


In [42]:
neighborhoods_venues_sorted.shape


(39, 12)